In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets,transforms

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])
x_train=datasets.MNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)
x_test=datasets.MNIST(
    root='./data',
    train=False,
    transform=transform,
    download=True
)
pin = torch.cuda.is_available()

train_loader=torch.utils.data.DataLoader(
    x_train,
    batch_size=128,
    shuffle=True,
    num_workers=4,
    pin_memory=pin
)
test_loader=torch.utils.data.DataLoader(
    x_test,
    batch_size=128,
    shuffle=False,
    num_workers=4,
    pin_memory=pin
)

class CNN(nn.Module):
  def __init__(self):
      super().__init__()
      self.conv1=nn.Conv2d(1,16,3)
      self.conv2=nn.Conv2d(16,32,3)
      self.pool=nn.MaxPool2d(2,2)
      self.relu=nn.ReLU()
      self.fc1=nn.Linear(32*5*5,64)
      self.fc2=nn.Linear(64,10)

  def forward(self,x):
    x=self.pool(self.relu(self.conv1(x)))
    x=self.pool(self.relu(self.conv2(x)))
    x=x.view(x.size(0),-1)
    x=self.relu(self.fc1(x))
    x=self.fc2(x)
    return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

epoch=3
for i in range(epoch):
  model.train()
  t_loss=0

  for images,labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    output=model(images)
    loss=loss_fn(output,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    t_loss+=loss.item()

  print(f"Epoch {i}:Loss={t_loss:.3f}")


model.eval()
correct=0
tot=0

with torch.no_grad():
    for images,labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      output=model(images)
      _,pred=torch.max(output,1)

      tot+=labels.size(0)
      correct+=(pred==labels).sum().item()
      #for p in pred:
           # print(f"Predicted digit: {p.item()}")

    print(f"Accuracy:{100*correct/tot:.3f}")

def predict_digit(image, model, device):
    model.eval()
    with torch.no_grad():
        if isinstance(image, np.ndarray):
            image = torch.tensor(image, dtype=torch.float32)
            image = (image - 0.1307) / 0.3081  # normalize
        image = image.unsqueeze(0).unsqueeze(0).to(device)
        output = model(image)
        _, pred = torch.max(output, 1)
        return pred.item()


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 0:Loss=113.832
Epoch 1:Loss=30.700
Epoch 2:Loss=21.397
Accuracy:98.630
